In [5]:
from __future__ import print_function

import torch
import torchvision.transforms as transforms

import torch.backends.cudnn as cudnn
from networks.resnet_big import SupConResNet
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score, f1_score

model_path = './save/SupCon/path_models/SupCon_path_resnet50_lr_0.0005_decay_0.0001_bsz_128_temp_0.1_trial_0_cosine/ckpt_epoch_100.pth'
model = SupConResNet(name='resnet50')
model.feat = torch.nn.Linear()

state_dict = torch.load(model_path)


print("Keys in the state dictionary:")
for key in state_dict.keys():
    print(key)


Keys in the state dictionary:
opt
model
optimizer
epoch


In [28]:
for name, module in model.named_modules():
    if isinstance(module, torch.nn.Conv2d):  # 检查是否为卷积层
        print(f"{name} weights:", module.weight.data.size())

encoder.conv1 weights: torch.Size([64, 3, 3, 3])
encoder.layer1.0.conv1 weights: torch.Size([64, 64, 1, 1])
encoder.layer1.0.conv2 weights: torch.Size([64, 64, 3, 3])
encoder.layer1.0.conv3 weights: torch.Size([256, 64, 1, 1])
encoder.layer1.0.shortcut.0 weights: torch.Size([256, 64, 1, 1])
encoder.layer1.1.conv1 weights: torch.Size([64, 256, 1, 1])
encoder.layer1.1.conv2 weights: torch.Size([64, 64, 3, 3])
encoder.layer1.1.conv3 weights: torch.Size([256, 64, 1, 1])
encoder.layer1.2.conv1 weights: torch.Size([64, 256, 1, 1])
encoder.layer1.2.conv2 weights: torch.Size([64, 64, 3, 3])
encoder.layer1.2.conv3 weights: torch.Size([256, 64, 1, 1])
encoder.layer2.0.conv1 weights: torch.Size([128, 256, 1, 1])
encoder.layer2.0.conv2 weights: torch.Size([128, 128, 3, 3])
encoder.layer2.0.conv3 weights: torch.Size([512, 128, 1, 1])
encoder.layer2.0.shortcut.0 weights: torch.Size([512, 256, 1, 1])
encoder.layer2.1.conv1 weights: torch.Size([128, 512, 1, 1])
encoder.layer2.1.conv2 weights: torch.Si

In [7]:
model.load_state_dict(state_dict['model'])
model.eval()
  

SupConResNet(
  (encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [8]:
from torchvision import transforms, datasets
from torch.utils.data import DataLoader


data_transforms = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop((224, 224)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


data_dir = './datasets/PaHaW_visualization'
dataset = datasets.ImageFolder(root=data_dir, transform=data_transforms)
dataloader = DataLoader(dataset, batch_size=4, shuffle=False)



In [20]:
print(dataset.classes)

['Health', 'PD']


In [32]:
from sklearn.metrics import f1_score, accuracy_score,precision_score,recall_score
import numpy as np
from sklearn.linear_model import LogisticRegression

features = []
true_labels = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device) 
        output = model(inputs)
        features.extend(output.cpu().numpy()) 
        true_labels.extend(labels.cpu().numpy())


features = np.array(features)
true_labels = np.array(true_labels)


classifier = LogisticRegression(max_iter=1000)
classifier.fit(features, true_labels)

pred_labels = classifier.predict(features)

In [35]:
accuracy = accuracy_score(true_labels, pred_labels)
f1 = f1_score(true_labels, pred_labels, average="binary")
precision = precision_score(true_labels, pred_labels, average="binary")
recall = recall_score(true_labels, pred_labels, average="binary")

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

Accuracy: 0.5277777777777778
F1 Score: 0.5641025641025642
Precision: 0.5238095238095238
Recall: 0.6111111111111112
